In [1]:
import plotly
import plotly.express as px
import plotly.graph_objects as go
import dash
import jupyter_dash as jd
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input, State
from dash.exceptions import PreventUpdate
from dash_table import DataTable
import pandas as pd
pd.options.display.max_columns = None

for p in [plotly, dash, jd, dcc, html, dbc, pd,]:
    print(f'{p.__name__:-<30}v{p.__version__}')

ModuleNotFoundError: No module named 'plotly'

In [2]:
df = pd.read_csv('data/2019.csv')
df['num_chars'] = df['full_text'].str.len()
df['num_words'] = df['full_text'].str.split().str.len()



# Top words by freq

word_tweet_freq=[]
for tweet in df['full_text']:
    for words in tweet.split():
        word_tweet_freq.append(words)
word_fre = FreqDist(word_tweet_freq)

# create a Df with th efrequency of each word
df_words = pd.DataFrame.from_dict(dict_to_df)

app = JupyterDash(__name__)

app.layout = html.Div([

    dbc.Label('EDA I'),
    dcc.Dropdown(id='hist_kind_dropdown',
                 options=[{'label': 'num_words', 'value': 'num_words'},
                          {'label': 'num_chars', 'value': 'num_chars'}]),
    dcc.Graph(id='freq_hist')
    ])

@app.callback(Output('freq_hist', 'figure'),
              Input('hist_kind_dropdown', 'value'))
def plot_hist(kind):
    if kind == 'num_words':
        fig = px.histogram(df,
                           x = 'num_words')
        return fig
        


    if kind == 'num_chars':
        fig = px.histogram(df,
                           x = 'num_chars')
        return fig


app.run_server(debug=True, port=8041)

ModuleNotFoundError: No module named 'jupyterDash'